# Baseline Model Training

This notebook trains a baseline TF-IDF + Logistic Regression model for financial sentiment classification.


In [ ]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from dataset_loader import load_dataset
from preprocess import preprocess_batch
from model import build_model, get_all_top_features

%matplotlib inline
plt.style.use('seaborn-v0_8')


## Load and Prepare Data


In [ ]:
# Load dataset
# data_path = 'data/financial_phrasebank.csv'  # Update with your path
# dataset_name = 'phrasebank'
# 
# df = load_dataset(dataset_name, data_path)
# print(f"Loaded {len(df)} samples")
# 
# # Preprocess
# df['cleaned_text'] = preprocess_batch(df['text'])
# df = df[df['cleaned_text'].str.len() > 0]
# 
# # Split train/test
# X = df['cleaned_text'].values
# y = df['label'].values
# 
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )
# 
# print(f"Train: {len(X_train)}, Test: {len(X_test)}")


## Build and Train Model


In [ ]:
# Build model
# model = build_model(max_features=10000, ngram_range=(1, 2))
# 
# # Train
# print("Training model...")
# model.fit(X_train, y_train)
# print("Training completed!")


## Evaluate Model


In [ ]:
# Make predictions
# y_pred = model.predict(X_test)
# y_proba = model.predict_proba(X_test)
# 
# # Accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy:.4f}")
# 
# # Classification report
# print("\nClassification Report:")
# print(classification_report(y_test, y_pred))
# 
# # Confusion matrix
# cm = confusion_matrix(y_test, y_pred)
# plt.figure(figsize=(10, 8))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#             xticklabels=model.named_steps['classifier'].classes_,
#             yticklabels=model.named_steps['classifier'].classes_)
# plt.title('Confusion Matrix')
# plt.ylabel('True Label')
# plt.xlabel('Predicted Label')
# plt.tight_layout()
# plt.show()


## Top Features


In [ ]:
# Get top features for each class
# top_features = get_all_top_features(model, top_n=20)
# 
# for class_name, features in top_features.items():
#     print(f"\n{class_name.upper()}:")
#     for feature, weight in features[:10]:
#         print(f"  {feature:20s} {weight:8.4f}")
